<a href="https://colab.research.google.com/github/claudiaqw/deep-learning/blob/main/hungerGames_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab assignment: the hunger games

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

In this assignment we will face a challenging image classification problem, building a deep learning model that is able to classify different kinds of foods. Let the hunger games begin!

## Guidelines

Throughout this notebook you will find empty cells that you will need to fill with your own code. Follow the instructions in the notebook and pay special attention to the following symbols.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
You will need to solve a question by writing your own code or answer in the cell immediately below or in a different file, as instructed.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
This is a hint or useful observation that can help you solve this assignment. You should pay attention to these hints to better understand the assignment.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
This is an advanced exercise that can help you gain a deeper knowledge into the topic. Good luck!</font>

***

To avoid missing packages and compatibility issues you should run this notebook under one of the [recommended Deep Learning environment files](https://github.com/albarji/teaching-environments/tree/master/deeplearning), or make use of [Google Colaboratory](https://colab.research.google.com/). If you use Colaboratory make sure to [activate GPU support](https://colab.research.google.com/notebooks/gpu.ipynb).

The following code will embed any plots into the notebook instead of generating a new window:

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

Lastly, if you need any help on the usage of a Python function you can place the writing cursor over its name and press Caps+Shift to produce a pop-out with related documentation. This will only work inside code cells. 

Let's go!

## Data acquisition

We will use a food images dataset available at [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). To download it you will need to create a user account in Kaggle, and obtain your API credential by following the instructions on [this section](https://www.kaggle.com/trolukovich/food11-image-dataset). Once you have your credentials JSON file, you can inform this notebook of them by setting the appropriate enviroment variables, as follows

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Once this is done, you will be able to download the dataset to this computer using the command

    !kaggle datasets download trolukovich/food11-image-dataset --unzip -p YOUR_LOCAL_FOLDER
    
where you should write a valid directory in your computer in "YOUR_LOCAL_FOLDER". If you are fine with downloading the data in the same folder as this notebook, just skip the `-p YOUR_LOCAL_FOLDER` part of the command.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Create a Kaggle account, obtain your credentials, and use the cell below to declare your Kaggle username and key variables, and to download the dataset to a local folder.
    
These credentials should remain secret to you. Remember to delete them before submitting this notebook!
</font>

***

In [2]:
####### INSERT YOUR CODE HERE

import os
credentials = {"username":"claudiaqw",
               "key":"7a4a42e84feb6a7da837ebf98550b12b"}

os.environ['KAGGLE_USERNAME']=credentials["username"]
os.environ['KAGGLE_KEY']=credentials["key"]

!kaggle datasets download trolukovich/food11-image-dataset --unzip

 98% 1.06G/1.08G [00:09<00:00, 93.0MB/s]
100% 1.08G/1.08G [00:09<00:00, 118MB/s] 


Take now a look into the folder where you downloaded the data. You will find it is made up of three subfolders:

* **training**, containing the images to use to train the model.
* **validation**, containing additional images you could use as more training data, or for some kind of validation strategy such as Early Stopping.
* **evaluation**, containing the images you must use to test your model. Images in this folder can **only** be used to measure the model performance after the training procedure is completed.

Furthermore, within each one of these folders you will find one folder for each one of the 11 classes in this problem:

* Bread
* Dairy product
* Dessert
* Egg
* Fried food
* Meat
* Noodles-Pasta
* Rice
* Seafood
* Soup
* Vegetable-Fruit

This is a standard structure for organizing image datasets: one folder per class. To easen the following data processing steps, let us define some variables telling us where the data is located.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Create variables <b>TRAINDIR</b>, <b>VALDIR</b> and <b>TESTDIR</b> with the paths to the folders with the training, validation and evaluation data, respectively.
</font>

***

In [7]:
####### INSERT YOUR CODE HERE

TRAINDIR, VALDIR, TESTDIR = '/content/training', '/content/validation', '/content/evaluation'


Let's plot a random sample of training images from each class, using the ipyplot package. If you are running this notebook in Google Colab, you will need to install this package first with

    !pip install ipyplot

You can inspect each class by clicking the different tabs in the interface that will appear when running the following cell.

In [4]:
#!pip install ipyplot

In [ ]:
from glob import glob
import ipyplot
import numpy as np

all_images = glob(f"{TRAINDIR}/*/*.jpg")  # Get all image paths
np.random.shuffle(all_images)  # Randomize to show different images each run
all_labels = [f.split("/")[-2] for f in all_images]  # Extract class names from path

ipyplot.plot_class_tabs(all_images, all_labels, max_imgs_per_tab=6, img_width=300, force_b64=True)

### Class reduction

To make the problem more approachable for this exercise, we will focus on just six classes: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` and `Meat`. To do so, we will delete from the downloaded data the folders from other classes.

In [8]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

Deleting /content/validation/Soup...
Deleting /content/validation/Seafood...
Deleting /content/validation/Vegetable-Fruit...
Deleting /content/validation/Noodles-Pasta...
Deleting /content/validation/Rice...
Deleting /content/evaluation/Soup...
Deleting /content/evaluation/Seafood...
Deleting /content/evaluation/Vegetable-Fruit...
Deleting /content/evaluation/Noodles-Pasta...
Deleting /content/evaluation/Rice...
Deleting /content/training/Soup...
Deleting /content/training/Seafood...
Deleting /content/training/Vegetable-Fruit...
Deleting /content/training/Noodles-Pasta...
Deleting /content/training/Rice...


## Image processing from files

This dataset of images is large, with images of larger resolution than in the tutorial MNIST problem, each one having different sizes and image ratios. Also, while for MNIST we had a keras function that prepared the dataset for us, this time we will need to do some loading and image processing work.

A convenient way to do this work is through the use of Keras `image_dataset_from_directory` function. This function creates a TensorFlow `Dataset` with the images in the directory, loading images dynamically only when the neural network needs to use them, and also allowing us to specify some useful preprocessing options.

For example, we can create such a `Dataset` with the data in the training folder:

In [9]:
from keras.preprocessing import image_dataset_from_directory

image_size = 32
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Found 6082 files belonging to 6 classes.


Note the parameters used to configure the dataset:

* The **directory** from which to load the images.
* An **image size** that will be used to resize all the images to a common resolution, here 32x32.
* The **size of the batches** of images to be generated. Note we define this parameter here instead in the network fit step, as the `Dataset` will make use of this information to keep in memory only a few batches of images at the same time in order to save memory.
* The **label mode**, that is, the encoding used for the labels. `categorical` means we will use one-hot encoding.

A `Dataset` object works like a Python generator, which means we can iterate over it to obtain batches of processed images. For instance, to get the first batch

In [ ]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

We can see that indeed the generator produces a tensor of the appropriate dimensions with the inputs for the neural network, and that the outputs have also been properly codified in one-hot form. However, there is still an issue with the data: the pixel values are in the range [0, 255], which might produce training problems. We will solve this later in the network definition. For now, let's move on and prepare a funcion that builds the training, validation and test datasets.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Create a function <b>create_datasets</b> that receives the following parameters:
    <ul>
      <li><b>traindir</b>: the directory where training images are located.</li>
      <li><b>valdir</b>: the directory where validation images are located.</li>
      <li><b>testdir</b>: the directory where test images are located.</li>
      <li><b>image_size</b>: the size that will be used to resize all the images to a common resolution.</li>
      <li><b>batch_size</b>: the size of the batches of images to be generated.</li>
    </ul>
    The function must create datasets for the training, validation and test directories, and return the three datasets created.

    return train_dataset, val_dataset, test_dataset
</font>

***

In [10]:
####### INSERT YOUR CODE HERE

def create_datasets(traindir=TRAINDIR, valdir=VALDIR, testdir=TESTDIR, image_size=32, batch_size=64):
  label_mode = 'categorical'
  
  train_dataset = image_dataset_from_directory(
    traindir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )
  
  val_dataset = image_dataset_from_directory(
    valdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )
  
  test_dataset = image_dataset_from_directory(
    testdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )
  
  return train_dataset, val_dataset, test_dataset

Let's test if the function you just implemented works correctly

In [11]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


Now that we have our datasets we can train a deep learning model using them! For illustration purposes, let's build an extremely simple convolutional network. Note how we have added a special pre-processing layer `Rescaling` that takes care of normalizing the data to the range [0, 1].

Be careful! This network will work, but has some flaws in its design you might want to fix in the network you will desing later in this notebook.

In [12]:
from numpy.random import seed
import random

seed(42)
tf.random.set_seed(42)

In [13]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.experimental.preprocessing import Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

The `fit` method of a Keras model can receive a `Dataset` with training data, instead of a pair of tensors with (inputs, outputs). Since when building the `Dataset` we already specified the batch size, we don't need to do it now.

In [14]:
model.fit(train_dataset, epochs=1)

96/96 [==============================] - 43s 137ms/step - loss: 1.7418 - accuracy: 0.2497


Similarly, we can evaluate the performance of our model over our test `Dataset` as follows

In [15]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 130ms/step - loss: 2.0489 - accuracy: 0.2522
Loss 2.05, accuracy 25.2%


The accuracy might seem poor, but take into account we have used a very simple model and this problem has 6 classes. Will you be able to do better?

## Building your network

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Design a neural network that maximizes the accuracy over the test data. You can use the training and validation datasets for anything you like, but you can <b>only</b> use the test data to evaluate the model performance. You should obtain a network able to attain at least 40% accuracy over the test set.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Some tips and strategies that can help you optimize your network design:

    
- Make use of all the tricks you learned from previous notebooks: convolutional + pooling layers, ReLU activations, dropout... also make sure to use a good optimizer with an adequate loss function, as well as the correct activation for the output layer.
- Configuring the datasets to load the images with a larger size can significantly improve your performance. But be careful, you can also run out of memory (CUDA memory error) if they become too large! Note that for this problem a size larger than 256 might be too large.
- Start with networks with a small number of parameters, so you are able to check fast how well they work. Then you can make your network larger in three directions: larger input images, more layers and more kernels per convolutional layer or units per dense layer. If you use larger images make sure to add more Convolution+Pooling layers, so that only very small images (less than 10x10 pixels) arrive at the Flatten layer.
- If you see large differences in loss between your training data and your validation or test data, try increasing the Dropout probabilities, especially for the Dense layers.
- Make use of that validation data! For instance, use an <a href="https://keras.io/api/callbacks/early_stopping/">**EarlyStopping strategy**</a> to monitor the loss of these validation data, and stop when training when after a number of iterations such loss has not decreased. Configuring the EarlyStopping to restore the best weights found in the optimization is also useful.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Other advanced strategies you can try are:

- Use **image augmentation techniques** to artifically create new training images. To do so, explore the rest of layers available in the <a href="https://keras.io/api/layers/preprocessing_layers/image_preprocessing/">Keras Image Preprocessing module</a>.
- Use <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> layers to improve the optimization procedure.
    
It you use all the tricks, it is possible to obtain more than 60% accuracy in the test set.

</font>

***

### First model: Changing optimizer

The first change in the original model is to replace the *sgd* optimizer by the *Adam*. As we can see, the model performance on test has improved.

In [16]:
seed(42)
tf.random.set_seed(42)

In [17]:
####### INSERT YOUR CODE HERE
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam

pool_size = 2

model_1 = Sequential()
model_1.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_1.add(Convolution2D(4, 3, activation='linear'))
model_1.add(Flatten())
model_1.add(Dense(6, activation='sigmoid'))

opt = Adam(learning_rate=0.01)
model_1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [18]:
model_1.fit(train_dataset, epochs=1)

96/96 [==============================] - 17s 160ms/step - loss: 2.4922 - accuracy: 0.2385


In [19]:
loss, acc = model_1.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 137ms/step - loss: 1.7244 - accuracy: 0.3256
Loss 1.72, accuracy 32.6%


### Second model: adding max pooling layer

In [20]:
seed(42)
tf.random.set_seed(42)

In [24]:
pool_size = 2

model_2 = Sequential()
model_2.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_2.add(Convolution2D(4, 3, activation='linear'))
model_2.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_2.add(Flatten())
model_2.add(Dense(6, activation='sigmoid'))

opt = Adam(learning_rate=0.01)
model_2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [22]:
model_2.fit(train_dataset, epochs=10)

96/96 [==============================] - 17s 158ms/step - loss: 1.8477 - accuracy: 0.2595


In [23]:
loss, acc = model_2.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 141ms/step - loss: 1.6033 - accuracy: 0.3493
Loss 1.6, accuracy 34.9%


### Third model: adding another convolution but with relu activation, a rectified linear activation and dropout with probability 0.5





Now, we try to improve the network adding another convolution with 32 channels to generate and an relu activation, a dense layer with 50 units and rectified linear activation and a dropout with 50% of probability

In [27]:
seed(42)
tf.random.set_seed(42)

In [ ]:
kernel_size = 3
model_3 = Sequential()
model_3.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_3.add(Convolution2D(4, (kernel_size, kernel_size), input_shape=(image_size, image_size, 3), activation='linear'))
model_3.add(Convolution2D(32, (kernel_size, kernel_size), activation="relu"))
model_3.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_3.add(Dense(50, activation="relu"))
model_3.add(Dropout(0.5))
model_3.add(Flatten())
model_3.add(Dense(6, activation='sigmoid'))

opt = Adam(learning_rate=0.01)
model_3.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
model_3.fit(train_dataset, epochs=10)

Epoch 1/10
96/96 [==============================] - 14s 136ms/step - loss: 1.8523 - accuracy: 0.2556
Epoch 2/10
96/96 [==============================] - 14s 134ms/step - loss: 1.5526 - accuracy: 0.3514
Epoch 3/10
96/96 [==============================] - 13s 127ms/step - loss: 1.5708 - accuracy: 0.3502
Epoch 4/10
96/96 [==============================] - 13s 128ms/step - loss: 1.5821 - accuracy: 0.3539
Epoch 5/10
96/96 [==============================] - 13s 125ms/step - loss: 1.5140 - accuracy: 0.3748
Epoch 6/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4610 - accuracy: 0.4019
Epoch 7/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4637 - accuracy: 0.4084
Epoch 8/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4913 - accuracy: 0.3979
Epoch 9/10
96/96 [==============================] - 13s 128ms/step - loss: 1.4761 - accuracy: 0.3991
Epoch 10/10
96/96 [==============================] - 13s 127ms/step - loss: 1.4976 - accura

In [ ]:
loss, acc = model_3.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 89ms/step - loss: 1.5514 - accuracy: 0.3478
Loss 1.55, accuracy 34.8%


As we can see, although the accuracy in train has decreased slightly, we have achieved a better one in test. 

### Fourth model: we decided to try with larger input images

In [57]:
train_dataset_64, val_dataset_64, test_dataset_64 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=64, batch_size=64)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [34]:
seed(42)
tf.random.set_seed(42)

In [35]:
kernel_size = 3
image_size = 64
model_4 = Sequential()
model_4.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_4.add(Convolution2D(4, (kernel_size, kernel_size), input_shape=(image_size, image_size, 3), activation='linear'))
model_4.add(Convolution2D(32, (kernel_size, kernel_size), activation="relu"))
model_4.add(MaxPooling2D(pool_size=pool_size, strides=2))
model_4.add(Dense(50, activation="relu"))
model_4.add(Dropout(0.5))
model_4.add(Flatten())
model_4.add(Dense(6, activation='sigmoid'))

opt = Adam(learning_rate=0.01)
model_4.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
model_4.fit(train_dataset_64, epochs=10)

Epoch 1/10
96/96 [==============================] - 14s 127ms/step - loss: 2.6222 - accuracy: 0.1988
Epoch 2/10
96/96 [==============================] - 14s 133ms/step - loss: 1.7344 - accuracy: 0.2411
Epoch 3/10
96/96 [==============================] - 14s 135ms/step - loss: 1.7311 - accuracy: 0.2402
Epoch 4/10
96/96 [==============================] - 14s 134ms/step - loss: 1.7304 - accuracy: 0.2443
Epoch 5/10
96/96 [==============================] - 14s 134ms/step - loss: 1.7313 - accuracy: 0.2454
Epoch 6/10
96/96 [==============================] - 14s 133ms/step - loss: 1.7306 - accuracy: 0.2416
Epoch 7/10
96/96 [==============================] - 14s 134ms/step - loss: 1.7332 - accuracy: 0.2403
Epoch 8/10
96/96 [==============================] - 14s 135ms/step - loss: 1.7299 - accuracy: 0.2454
Epoch 9/10
96/96 [==============================] - 14s 134ms/step - loss: 1.7292 - accuracy: 0.2447
Epoch 10/10
96/96 [==============================] - 14s 134ms/step - loss: 1.7178 - accura

In [ ]:
loss, acc = model_4.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 5s 122ms/step - loss: 1.7360 - accuracy: 0.2415
Loss 1.74, accuracy 24.2%


### Fifth Model:

This we are going to ...

This model reaches a high performance over training set, but not on validation wich means is overfitted

In [44]:
seed(42)
tf.random.set_seed(42)

In [45]:
model_5 = Sequential()
model_5.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_5.add(Convolution2D(4, 2, input_shape=(image_size, image_size, 3), activation='linear'))
model_5.add(MaxPooling2D(pool_size=pool_size, strides=1, padding='valid'))
model_5.add(Dropout(0.5))
model_5.add(Convolution2D(16, 2, activation='relu'))
model_5.add(MaxPooling2D(pool_size=pool_size, strides=1, padding='valid'))
model_5.add(Dropout(0.5))
model_5.add(Convolution2D(32, 2, activation='linear'))
model_5.add(MaxPooling2D(pool_size=pool_size, strides=1, padding='valid'))
# model_4.add(Dropout(0.5))
model_5.add(Flatten())
model_5.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.01)
model_5.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [46]:
model_5.fit(train_dataset_64, epochs=10)

Epoch 1/10
96/96 [==============================] - 17s 159ms/step - loss: 37.8502 - accuracy: 0.2205
Epoch 2/10
96/96 [==============================] - 17s 163ms/step - loss: 1.9058 - accuracy: 0.3224
Epoch 3/10
96/96 [==============================] - 17s 162ms/step - loss: 1.6885 - accuracy: 0.3800
Epoch 4/10
96/96 [==============================] - 17s 163ms/step - loss: 1.8685 - accuracy: 0.3746
Epoch 5/10
96/96 [==============================] - 17s 168ms/step - loss: 1.5763 - accuracy: 0.3998
Epoch 6/10
96/96 [==============================] - 17s 169ms/step - loss: 1.4071 - accuracy: 0.4487
Epoch 7/10
96/96 [==============================] - 17s 169ms/step - loss: 1.3393 - accuracy: 0.4766
Epoch 8/10
96/96 [==============================] - 17s 165ms/step - loss: 1.3603 - accuracy: 0.4710
Epoch 9/10
96/96 [==============================] - 17s 165ms/step - loss: 1.2411 - accuracy: 0.5100
Epoch 10/10
96/96 [==============================] - 17s 167ms/step - loss: 1.1827 - accur

In [47]:
loss, acc = model_5.evaluate(val_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 129ms/step - loss: 1.9813 - accuracy: 0.2386
Loss 1.98, accuracy 23.9%


In [48]:
loss, acc = model_5.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 141ms/step - loss: 1.9753 - accuracy: 0.2493
Loss 1.98, accuracy 24.9%


### Model 6

In [49]:
seed(42)
tf.random.set_seed(42)

In [50]:
model_6 = Sequential()
model_6.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_6.add(Convolution2D(4, 2, input_shape=(image_size, image_size, 3), activation='relu'))
model_6.add(MaxPooling2D(pool_size=pool_size, strides=1, padding='valid'))
model_6.add(Dropout(0.5))
model_6.add(Convolution2D(16, 2, activation='relu'))
model_6.add(MaxPooling2D(pool_size=pool_size, strides=1, padding='valid'))
model_6.add(Dropout(0.5))
model_6.add(Convolution2D(32, 2, activation='relu'))
model_6.add(MaxPooling2D(pool_size=pool_size, strides=1, padding='valid'))
# model_4.add(Dropout(0.5))
model_6.add(Flatten())
model_6.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.01)
model_6.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [51]:
model_6.fit(train_dataset_64, epochs=10)

Epoch 1/10
96/96 [==============================] - 20s 189ms/step - loss: 23.0424 - accuracy: 0.2116
Epoch 2/10
96/96 [==============================] - 19s 185ms/step - loss: 1.7336 - accuracy: 0.2409
Epoch 3/10
96/96 [==============================] - 19s 187ms/step - loss: 1.7290 - accuracy: 0.2443
Epoch 4/10
96/96 [==============================] - 19s 187ms/step - loss: 1.7343 - accuracy: 0.2409
Epoch 5/10
96/96 [==============================] - 19s 189ms/step - loss: 1.7346 - accuracy: 0.2394
Epoch 6/10
96/96 [==============================] - 19s 186ms/step - loss: 1.7325 - accuracy: 0.2394
Epoch 7/10
96/96 [==============================] - 19s 186ms/step - loss: 1.7292 - accuracy: 0.2411
Epoch 8/10
96/96 [==============================] - 19s 187ms/step - loss: 1.7302 - accuracy: 0.2412
Epoch 9/10
96/96 [==============================] - 19s 180ms/step - loss: 1.7302 - accuracy: 0.2446
Epoch 10/10
96/96 [==============================] - 19s 181ms/step - loss: 1.7319 - accur

In [52]:
loss, acc = model_6.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 7s 153ms/step - loss: 1.7360 - accuracy: 0.2415
Loss 1.74, accuracy 24.2%


### MODEL 7


In [ ]:
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam

In [ ]:
train_dataset_64, val_dataset_64, test_dataset_64 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=64, batch_size=64)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [ ]:
kernel_size = 3
image_size = 64
model_7 = Sequential()
model_7.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_7.add(Convolution2D(4, (kernel_size, kernel_size), input_shape=(image_size, image_size, 3)))
model_7.add(Dense(50, activation="relu"))
model_7.add(Convolution2D(4, (kernel_size, kernel_size)))
model_7.add(Dense(50, activation="relu"))
model_7.add(MaxPooling2D(pool_size=(2, 2), strides=1))
model_7.add(Convolution2D(8, (kernel_size, kernel_size)))
model_7.add(Dense(50, activation="relu"))
model_7.add(Convolution2D(8, (kernel_size, kernel_size)))
model_7.add(Dense(50, activation="relu"))
model_7.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_7.add(Convolution2D(16, (kernel_size, kernel_size)))
model_7.add(Dense(50, activation="relu"))
model_7.add(Convolution2D(16, (kernel_size, kernel_size)))
model_7.add(Dense(50, activation="relu"))
model_7.add(MaxPooling2D(pool_size=(5, 5), strides=3))
model_7.add(Flatten())
model_7.add(Dense(6, activation='sigmoid'))

opt = Adam(learning_rate=0.01)
model_7.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
model_7.fit(train_dataset_64, epochs=10)

Epoch 1/10
96/96 [==============================] - 20s 189ms/step - loss: 1.7924 - accuracy: 0.2142
Epoch 2/10
96/96 [==============================] - 19s 186ms/step - loss: 1.7464 - accuracy: 0.2388
Epoch 3/10
96/96 [==============================] - 19s 186ms/step - loss: 1.7362 - accuracy: 0.2370
Epoch 4/10
96/96 [==============================] - 19s 186ms/step - loss: 1.7299 - accuracy: 0.2444
Epoch 5/10
96/96 [==============================] - 19s 185ms/step - loss: 1.7427 - accuracy: 0.2368
Epoch 6/10
96/96 [==============================] - 19s 186ms/step - loss: 1.7321 - accuracy: 0.2402
Epoch 7/10
96/96 [==============================] - 19s 184ms/step - loss: 1.7305 - accuracy: 0.2443
Epoch 8/10
96/96 [==============================] - 19s 185ms/step - loss: 1.7315 - accuracy: 0.2454
Epoch 9/10
96/96 [==============================] - 19s 185ms/step - loss: 1.7307 - accuracy: 0.2416
Epoch 10/10
96/96 [==============================] - 19s 185ms/step - loss: 1.7333 - accura

In [ ]:
loss, acc = model_7.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 140ms/step - loss: 1.7358 - accuracy: 0.2415
Loss 1.74, accuracy 24.2%


### Model 8

In [95]:
image_size = 64
model_8 = Sequential()
model_8.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_8.add(Convolution2D(4, 3, input_shape=(image_size, image_size, 3), strides=2, activation= 'relu'))
model_8.add(Convolution2D(8, 3,strides=1, activation= 'relu'))
model_8.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_8.add(Convolution2D(16, 3,strides=1, activation= 'relu'))
model_8.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model_8.add(Flatten())
model_8.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.001)
model_8.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [96]:
model_8.fit(train_dataset_64, epochs=20)

Epoch 1/20
96/96 [==============================] - 11s 106ms/step - loss: 1.7289 - accuracy: 0.2546
Epoch 2/20
96/96 [==============================] - 11s 106ms/step - loss: 1.5786 - accuracy: 0.3330
Epoch 3/20
96/96 [==============================] - 11s 107ms/step - loss: 1.5501 - accuracy: 0.3519
Epoch 4/20
96/96 [==============================] - 11s 107ms/step - loss: 1.4830 - accuracy: 0.3987
Epoch 5/20
96/96 [==============================] - 11s 107ms/step - loss: 1.4454 - accuracy: 0.4164
Epoch 6/20
96/96 [==============================] - 11s 106ms/step - loss: 1.4167 - accuracy: 0.4370
Epoch 7/20
96/96 [==============================] - 11s 104ms/step - loss: 1.4064 - accuracy: 0.4383
Epoch 8/20
96/96 [==============================] - 11s 106ms/step - loss: 1.3930 - accuracy: 0.4433
Epoch 9/20
96/96 [==============================] - 11s 106ms/step - loss: 1.3772 - accuracy: 0.4496
Epoch 10/20
96/96 [==============================] - 11s 105ms/step - loss: 1.3482 - accura

In [97]:
loss, acc = model_8.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 95ms/step - loss: 1.3564 - accuracy: 0.4700
Loss 1.36, accuracy 47.0%


In [56]:
model_8.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_11 (Rescaling)     (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 31, 31, 4)         112       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 29, 29, 8)         296       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 12, 12, 16)        1168      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 576)             

### Model

In [91]:
image_size = 64
model_10 = Sequential()
model_10.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_10.add(Convolution2D(4, 3, input_shape=(image_size, image_size, 3), strides=2, activation= 'relu'))
model_10.add(Convolution2D(8, 3,strides=1, activation= 'relu'))
model_10.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_10.add(Convolution2D(16, 3,strides=1, activation= 'relu'))
model_10.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu'))
model_10.add(Dropout(0.5))
model_10.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.001)
model_10.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [92]:
model_10.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_25 (Rescaling)     (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 31, 31, 4)         112       
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 29, 29, 8)         296       
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 12, 12, 16)        1168      
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 576)             

In [93]:
model_10.fit(train_dataset_64, epochs=50)

Epoch 1/50
96/96 [==============================] - 11s 106ms/step - loss: 1.7109 - accuracy: 0.2640
Epoch 2/50
96/96 [==============================] - 11s 108ms/step - loss: 1.5476 - accuracy: 0.3603
Epoch 3/50
96/96 [==============================] - 11s 109ms/step - loss: 1.4872 - accuracy: 0.3764
Epoch 4/50
96/96 [==============================] - 11s 106ms/step - loss: 1.4507 - accuracy: 0.4143
Epoch 5/50
96/96 [==============================] - 11s 107ms/step - loss: 1.4093 - accuracy: 0.4332
Epoch 6/50
96/96 [==============================] - 11s 106ms/step - loss: 1.3941 - accuracy: 0.4407
Epoch 7/50
96/96 [==============================] - 11s 107ms/step - loss: 1.3824 - accuracy: 0.4376
Epoch 8/50
96/96 [==============================] - 11s 108ms/step - loss: 1.3368 - accuracy: 0.4566
Epoch 9/50
96/96 [==============================] - 11s 107ms/step - loss: 1.3456 - accuracy: 0.4583
Epoch 10/50
96/96 [==============================] - 11s 107ms/step - loss: 1.3537 - accura

In [94]:
loss, acc = model_10.evaluate(test_dataset_64)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 91ms/step - loss: 1.7002 - accuracy: 0.5019
Loss 1.7, accuracy 50.2%


### Model 9: AlexNet

In [59]:
train_dataset_128, val_dataset_256, test_dataset_256 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=256, batch_size=32)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [85]:
image_size = 256
model_9 = Sequential()
model_9.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_9.add(Convolution2D(96, 7, input_shape=(image_size, image_size, 3), strides=1, activation= 'relu'))
model_9.add(Convolution2D(128, 5, strides=2, activation= 'relu'))
model_9.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_9.add(Convolution2D(256, 5, strides=1, activation= 'relu'))
model_9.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model_9.add(Convolution2D(256, 3, strides=1, activation= 'relu'))
model_9.add(Convolution2D(512, 3, strides=1, activation= 'relu'))
# model_9.add(Convolution2D(512, 3, strides=1, activation= 'relu'))
model_9.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model_9.add(Flatten())
model_9.add(Dense(512, activation='relu'))
model_9.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.001)
model_9.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [86]:
model_9.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_22 (Rescaling)     (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 250, 250, 96)      14208     
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 123, 123, 128)     307328    
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 61, 61, 128)       0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 57, 57, 256)       819456    
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 28, 28, 256)       0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 26, 26, 256)     

In [ ]:
model_9.fit(train_dataset_256, epochs=10)

In [ ]:
loss, acc = model_9.evaluate(test_dataset_256)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

### Model 11:

In [110]:
train_dataset_128, val_dataset_128, test_dataset_128 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=64)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [158]:
seed(42)
tf.random.set_seed(42)

In [167]:
image_size = 128
model_11 = Sequential()
model_11.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_11.add(Convolution2D(4, 3, input_shape=(image_size, image_size, 3), strides=1, activation= 'relu'))
model_11.add(MaxPooling2D(pool_size=(2, 2), strides=1))
model_11.add(Convolution2D(8, 3,strides=2, activation= 'relu'))
model_11.add(Dropout(0.5))
model_11.add(Convolution2D(16, 3,strides=1, activation= 'relu'))
model_11.add(MaxPooling2D(pool_size=3, strides=2))
model_11.add(Convolution2D(16, 3,strides=2, activation= 'relu'))
model_11.add(MaxPooling2D(pool_size=2, strides=1))
model_11.add(Flatten())
model_11.add(Dense(1024, activation='relu'))
model_11.add(Dropout(0.7))
model_11.add(Dense(256, activation='relu'))
model_11.add(Dropout(0.7))
model_11.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.01)
model_11.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [160]:
model_11.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_40 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_141 (Conv2D)          (None, 126, 126, 4)       112       
_________________________________________________________________
max_pooling2d_100 (MaxPoolin (None, 125, 125, 4)       0         
_________________________________________________________________
conv2d_142 (Conv2D)          (None, 62, 62, 8)         296       
_________________________________________________________________
dropout_24 (Dropout)         (None, 62, 62, 8)         0         
_________________________________________________________________
conv2d_143 (Conv2D)          (None, 60, 60, 16)        1168      
_________________________________________________________________
max_pooling2d_101 (MaxPoolin (None, 29, 29, 16)      

In [161]:
model_11.fit(train_dataset_128, epochs=20)

Epoch 1/20
96/96 [==============================] - 44s 448ms/step - loss: 2.0111 - accuracy: 0.2342
Epoch 2/20
96/96 [==============================] - 43s 444ms/step - loss: 1.7251 - accuracy: 0.2481
Epoch 3/20
96/96 [==============================] - 42s 442ms/step - loss: 1.6612 - accuracy: 0.2585
Epoch 4/20
96/96 [==============================] - 43s 449ms/step - loss: 1.6241 - accuracy: 0.2924
Epoch 5/20
96/96 [==============================] - 43s 448ms/step - loss: 1.5824 - accuracy: 0.3137
Epoch 6/20
96/96 [==============================] - 44s 455ms/step - loss: 1.5341 - accuracy: 0.3515
Epoch 7/20
96/96 [==============================] - 44s 455ms/step - loss: 1.5409 - accuracy: 0.3477
Epoch 8/20
96/96 [==============================] - 44s 459ms/step - loss: 1.5757 - accuracy: 0.3329
Epoch 9/20
96/96 [==============================] - 44s 456ms/step - loss: 1.4987 - accuracy: 0.3703
Epoch 10/20
96/96 [==============================] - 44s 460ms/step - loss: 1.4530 - accura

In [166]:
loss, acc = model_11.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 5s 107ms/step - loss: 1.4830 - accuracy: 0.3838
Loss 1.46, accuracy 40.0%


### Model: ImageGenerator and Early Stopping

In [163]:
from keras.preprocessing.image import ImageDataGenerator
def create_dataset_image_generator(traindir=TRAINDIR, valdir=VALDIR, testdir=TESTDIR, image_size=32, batch_size=64):
  label_mode = 'categorical'
  train_datagen = ImageDataGenerator(
    zoom_range=0.2,
    rotation_range = 5,
    horizontal_flip=True,
    )

  train_dataset = train_datagen.flow_from_directory(
    traindir,
    target_size = (image_size,image_size),
    class_mode='categorical',
    batch_size = batch_size,

  )

  val_dataset = image_dataset_from_directory(
    valdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )

  test_dataset = image_dataset_from_directory(
    testdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = label_mode
    )

  return train_dataset, val_dataset, test_dataset

In [168]:
train_dataset_128, val_dataset_128, test_dataset_128 = create_dataset_image_generator(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=64)

Found 6082 images belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [173]:
image_size = 128
model_12 = Sequential()
model_12.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model_12.add(Convolution2D(4, 3, input_shape=(image_size, image_size, 3), strides=1, activation= 'relu'))
model_12.add(MaxPooling2D(pool_size=(2, 2), strides=1))
model_12.add(Convolution2D(8, 3,strides=2, activation= 'relu'))
# model_12.add(Dropout(0.5))
model_12.add(Convolution2D(16, 3,strides=1, activation= 'relu'))
model_12.add(MaxPooling2D(pool_size=3, strides=2))
model_12.add(Convolution2D(16, 3,strides=2, activation= 'relu'))
model_12.add(MaxPooling2D(pool_size=2, strides=1))
model_12.add(Flatten())
model_12.add(Dense(1024, activation='relu'))
model_12.add(Dropout(0.5))
model_12.add(Dense(256, activation='relu'))
model_12.add(Dropout(0.5))
model_12.add(Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.01)
model_12.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [174]:
model_12.fit(train_dataset_128, epochs=20)

Epoch 1/20
96/96 [==============================] - 45s 464ms/step - loss: 1.8244 - accuracy: 0.2221
Epoch 2/20
96/96 [==============================] - 45s 472ms/step - loss: 1.7450 - accuracy: 0.2190
Epoch 3/20
96/96 [==============================] - 44s 459ms/step - loss: 1.7304 - accuracy: 0.2447
Epoch 4/20
96/96 [==============================] - 44s 460ms/step - loss: 1.7329 - accuracy: 0.2422
Epoch 5/20
96/96 [==============================] - 44s 461ms/step - loss: 1.7295 - accuracy: 0.2457
Epoch 6/20
96/96 [==============================] - 44s 461ms/step - loss: 1.7351 - accuracy: 0.2377
Epoch 7/20
96/96 [==============================] - 44s 458ms/step - loss: 1.7321 - accuracy: 0.2499
Epoch 8/20
96/96 [==============================] - 44s 464ms/step - loss: 1.7279 - accuracy: 0.2496
Epoch 9/20
96/96 [==============================] - 46s 477ms/step - loss: 1.7309 - accuracy: 0.2436
Epoch 10/20
96/96 [==============================] - 45s 469ms/step - loss: 1.7343 - accura

KeyboardInterrupt: ignored

In [ ]:
loss, acc = model_12.evaluate(test_dataset_128)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

In [ ]:
from tf.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='loss', patience=3)

# history = model_12.fit(train_dataset_128, epochs=20)

## Transfer learning

While designing your own network might produce some nice results, it is generally better to transfer the knowledge available in a pre-trained network. This not only can produce better results, but also saves a lot of design time. The [Keras Applications](https://keras.io/api/applications/) module contains several network designs ready to use. For instance, to exploit the famous VGG16 network we do

In [ ]:
from keras.applications import VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

58892288/58889256 [==============================] - 0s 0us/step


By default all Keras Applications networks are preloaded with the weights that were obtained from training the network over the [ImageNet dataset](http://www.image-net.org/). To adapt the network to our problem we need to specify the shape of our input images, and also remove the output layers (top) of the original network, since we have a different number of classes.

Now, how do we do transfer learning over this network? We will show here how to implement the bottleneck features strategy. First, we will mark the VGG16 model as **non-trainable**, so that the weights remain frozen

In [ ]:
vgg16_model.trainable = False

Now we will build a neural network that includes the VGG16 model as one of its "layers". Since the VGG16 was trained with an specific way of normalizing the images, we will need to normalize our images in the same way. Conveniently, Keras also provides a function for doing VGG16-style normalization.

In [ ]:
from keras.applications.vgg16 import preprocess_input

We can try this with some image ir our dataset

In [ ]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

Before normalizing: [[[13.75  9.75  6.75]
  [13.25 12.25 10.25]
  [13.25  9.25  8.25]]

 [[16.5  12.5   9.5 ]
  [15.75 10.75  7.75]
  [19.25 14.25 11.25]]

 [[16.25 12.25  9.25]
  [12.    8.    5.  ]
  [21.75 17.75 13.25]]]
After normalizing: [[[ -97.189 -107.029 -109.93 ]
  [ -93.689 -104.529 -110.43 ]
  [ -95.689 -107.529 -110.43 ]]

 [[ -94.439 -104.279 -107.18 ]
  [ -96.189 -106.029 -107.93 ]
  [ -92.689 -102.529 -104.43 ]]

 [[ -94.689 -104.529 -107.43 ]
  [ -98.939 -108.779 -111.68 ]
  [ -90.689  -99.029 -101.93 ]]]


The normalization required by VGG16 involves swapping the order of color channels (RGB -> BGR) and substracting the mean values over the ImageNet dataset for each color channel separately. Fortunately the `preprocess_input` function does all the work for us. Furthermore, we can plug this function as the first layer of our network, similarly to the `Rescaling` we used before. We can do this with a `Lambda` layer, which allows building a layer out of any (differentiable!) function. Let's start our model with this layer.

In [ ]:
from keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

After this, we can add the whole VGG16 network as a layer, and our custom trainable layers after it. Note this is an overly simple model with some mistakes introduced; a real transfer learning network would have a better design.

In [ ]:
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 64, 64, 3)         0         
_________________________________________________________________
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 12294     
Total params: 14,726,982
Trainable params: 12,294
Non-trainable params: 14,714,688
_________________________________________________________________


Notice how in the model summary we can see the whole network has millions of parameters, but since we have frozen the VGG16 part, only a few thousand parameters will be trained: those in the Dense layer.

We can now compile and train this model in the usual way

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

ValueError: ignored

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Using the bottleneck strategy, implement a network doing transfer learning from VGG16. If you do it properly, you should be able to obtain better results than with your previously designed network, at least a 80% of accuracy over the test set.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Some tips and strategies that can help you optimize your network design:

    
- Include one or more Dense layers with the appropriate activation functions before the output layer.
- Try using a [GlobalAveragePooling layer](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) instead of a Flatten layer. This layer computes an average of all pixel values for each channel, and sometimes performs better than a regular Flatten.
- And remember all the tricks from the previous exercise!
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
You can further improve the network accuracy with the following ideas

- Use the PRO strategies from the previous exercise.
- Try other pre-trained networks from <a href="https://keras.io/api/applications/">Keras Applications</a>, such as ResNet, Xception or EfficientNet.
- Use more advanced transfer learning strategies, like fine-tuning or a combination of bottleneck features and fine-tuning.
   
If you use all the tricks, it is possible to obtain more than 90% accuracy in the test set.

</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Summary of results

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Write in the following cell a short report explaining what network designs you tried, and what test accuracies you obtained. What worked and what didn't? What have you learned from this exercise?
</font>

***

Example results table

|Image processing|Neural network model|Training strategy|Test accuracy|
|----------------|--------------------|-----------------|-------------|
|Size 32x32, batch size 16|Convolutional(32) + Flatten + Dense(64)|Train from scratch|xx%|
|Size 64x64, batch size 32|VGG16 + Flatten + Dense(32)|Bottleneck features|yy%|
|...|...|...|...|